In [1]:
from models import *
from trainer import *
import torch
import sched
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.nn import CrossEntropyLoss,SmoothL1Loss,BCEWithLogitsLoss
import torch.optim as optim
import time
from time import sleep
from tqdm import tqdm
import os
from pathlib import Path
import matplotlib.pyplot as plt


transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),  #先四周填充0，图像随机裁剪成32*32
    transforms.RandomHorizontalFlip(),  #图像一半的概率翻转，一半的概率不翻转
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),  
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

works_num=4
download=True
batch_size = 128
dataset_path = '/data/'
trainset = torchvision.datasets.CIFAR10(root=dataset_path, train=True, download=download, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=works_num,pin_memory=True)
testset = torchvision.datasets.CIFAR10(root=dataset_path, train=False, download=download, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=works_num,pin_memory=True)
torch.cuda.empty_cache()

Files already downloaded and verified
Files already downloaded and verified


In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
num_classes = 10

cuda


In [ ]:
res9_relu = ResNet9()
simp_res9_relu = simp_ResNet9()
res18_relu = ResNet18()

losses_res9_relu, train_acces_res9_relu, test_acces_res9_relu = train(device=device, model=res9_relu, train_loader=trainloader, test_loader=testloader, num_classes=num_classes, max_epoch=50)
losses_simp_res9_relu, train_acces_simp_res9_relu, test_acces_simp_res9_relu = train(device=device, model=simp_res9_relu, train_loader=trainloader, test_loader=testloader, num_classes=num_classes, max_epoch=50)
losses_res18_relu, train_acces_res18_relu, test_acces_res18_relu = train(device=device, model=res18_relu, train_loader=trainloader, test_loader=testloader, num_classes=num_classes, max_epoch=50)

test(device=device, net=res9_relu, train_loader=trainloader, test_loader=testloader)
test(device=device, net=simp_res9_relu, train_loader=trainloader, test_loader=testloader)
test(device=device, net=res18_relu, train_loader=trainloader, test_loader=testloader)

torch.save(res9_relu.state_dict(), 'res9_relu_base.pth')
torch.save(simp_res9_relu.state_dict(), 'simp_res9_relu_base.pth')
torch.save(res18_relu.state_dict(), 'res18_relu_base.pth')

# 绘制 loss 曲线
plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
plt.plot(losses_res9_relu)
plt.title('Res9 ReLU Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')

plt.subplot(1, 3, 2)
plt.plot(losses_simp_res9_relu)
plt.title('Simplified Res9 ReLU Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')

plt.subplot(1, 3, 3)
plt.plot(losses_res18_relu)
plt.title('Res18 ReLU Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.tight_layout()
plt.savefig('losses.png')

# 绘制训练集准确率曲线
plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
plt.plot(train_acces_res9_relu)
plt.title('Res9 ReLU Training Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')

plt.subplot(1, 3, 2)
plt.plot(train_acces_simp_res9_relu)
plt.title('Simplified Res9 ReLU Training Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')

plt.subplot(1, 3, 3)
plt.plot(train_acces_res18_relu)
plt.title('Res18 ReLU Training Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.tight_layout()
plt.savefig('train_accuracy.png')

# 绘制测试集准确率曲线
plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
plt.plot(test_acces_res9_relu)
plt.title('Res9 ReLU Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')

plt.subplot(1, 3, 2)
plt.plot(test_acces_simp_res9_relu)
plt.title('Simplified Res9 ReLU Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')

plt.subplot(1, 3, 3)
plt.plot(test_acces_res18_relu)
plt.title('Res18 ReLU Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.tight_layout()
plt.savefig('test_accuracy.png')

del res9_relu, simp_res9_relu, res18_relu

In [3]:
train(device=device, model=simp_res9, train_loader=trainloader, test_loader=testloader, num_classes=num_classes, max_epoch=1, scheduler='CyclicLR', loss="SmoothL1Loss")
test(device=device, net=simp_res9, train_loader=trainloader, test_loader=testloader)

1/1|██████████| [70.62s/epoch, loss:0.06, lr:0.10500, trainacc:35.22, testacc:36.64]	


Accuracy of the network on train set: 35.268 %
Accuracy of the network on test set: 36.640 %
